In [288]:
import pandas as pd
import numpy as np
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
import logging
logging.basicConfig(level=logging.DEBUG)
import statsmodels.formula.api as sm
#Load the data
file0 = "/Users/apple/Workspace/intelliJ/jockey/20181205_2_history.csv"
file1 = "/Users/apple/Workspace/intelliJ/jockey/20181205_2_predict.csv"
dataset0 = pd.read_csv(file0)
header= list(dataset0)
dataset1=pd.read_csv(file1)



In [291]:
allHorse = dataset0['horseId'].unique() # all horseId
allPredict=[]
for idx, horseId in enumerate(allHorse):
   
    train_dataset=dataset0[dataset0['horseId']==horseId]
    X=train_dataset[['raceMeeting', 'raceClass','distance','course','draw','addedWeight','declaredHorseWeight','jockey']]
    y=train_dataset[['finishTime']]

    test_dataset=dataset1[dataset1['horseId']==horseId]
    X_test=test_dataset[['raceMeeting', 'raceClass','distance','course','draw','addedWeight','declaredHorseWeight','jockey']]

    X=pd.concat([X,X_test])

    #y=dataset0.iloc[:,12].values #finishTime

    # We create the preprocessing pipelines for both numeric and categorical data.
    numeric_features = ['raceClass', 'distance','draw','addedWeight','declaredHorseWeight']
    numeric_transformer = Pipeline(steps=[
        ('imputer', SimpleImputer(strategy='median')),
        ('scaler', StandardScaler())])

    categorical_features = ['course','jockey']
    categorical_transformer = Pipeline(steps=[
        ('imputer', SimpleImputer(strategy='constant', fill_value='missing')),
        ('onehot', OneHotEncoder(handle_unknown='ignore'))])

    preprocessor = ColumnTransformer(
        transformers=[
            ('num', numeric_transformer, numeric_features),
            ('cat', categorical_transformer, categorical_features)])


    X_arr=preprocessor.fit_transform(X)
    #dummy value trap
    dummyVariableTrap=[]
    offset = len(numeric_features) 
    for i, val in enumerate(categorical_features):
        dummyVariableTrap.append(offset + X[val].nunique() -1)
        offset = offset + X[val].nunique()

    X_arr=np.delete(X_arr,dummyVariableTrap,1)


    #X_new=np.append(arr=np.ones((np.size(X_arr,0),1),dtype=int),values=X_arr,axis=1)
    #feature = [0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19]
    #X_opt = X_new[:,feature]
    #X_train=X_opt[:-1,:]
    #regresssor_OLS=sm.OLS(endog=y,exog=X_train).fit()
    #regresssor_OLS.summary()

    from sklearn.metrics import r2_score
    X_opt = X_arr
    X_train=X_opt[:-1,:]
    reg = LinearRegression()
    reg.fit(X_train,y)
    y_train=reg.predict(X_train)
    y_train
    score =r2_score(y, y_train) 
    score
    X_predict=np.array([X_opt[-1,:]])
    X_predict
    y_predict= reg.predict(X_predict)
    allPredict.append(y_predict[0][0])  # will print a message to the console

df = pd.DataFrame(data=[allHorse,allPredict]).transpose()
df.columns=['horseId','finishTime']
df = df.sort_values(by=['finishTime'])
df

/Users/apple/miniconda3/lib/python3.6/site-packages/ipykernel_launcher.py:41: DeprecationWarning: in the future the special handling of scalars will be removed from delete and raise an error
